#  Using the Planetary Variables Subscriptions API

This Jupyter notebook goes over the useage of the Planetary Variables API. As a demonstration, the example of creating a subscription for Biomass Proxy and downloading some of the resulting TIFs is shown.

For more detailed information, you can see the [Planetary Variables API user guide](https://docs.vandersat.com/data_access/api_user_guide.html#data-subscriptions).

Updated Feb 2023

# Part 0 - Getting set up

### 0.1 Install the necessary packages in your environment using pip and the requirements.txt file

In [5]:
#pip install -r requirements.txt
!pip install requests_toolbelt

Looking in indexes: https://pypi.org/simple, https://pypi.prod.planet-labs.com/simple/
  Using cached requests_toolbelt-0.10.1-py2.py3-none-any.whl (54 kB)


### 0.2 Import the necessary packages for this notebook

In [6]:
import glob
import logging
import math
import os
from typing import Iterator, List
import geopandas as gpd
import json
import time
import sys

from requests import Response
from requests_toolbelt.downloadutils import stream
from requests_toolbelt.exceptions import StreamingError
from requests_toolbelt.sessions import BaseUrlSession


### 0.3 API Functions that Run in the Background

In [7]:
def response_hook(response: Response, *_args, **_kwargs):
    """Hook to get detailed error details from the response body."""
    if response.status_code >= 400:
        logging.error(
            f"Error invoking API: url={response.url}; code={response.status_code}; "
            f"reason={response.reason}; message={response.text}"
        )

        exit(response.status_code)

### 0.4 Enter your credentials for the Planetary Variables API
It is recommended you do this via the use of environment variables

In [8]:
AUTH = (os.getenv("VDS_USER"), os.getenv("VDS_PASS"))

# Part 1 - Creating a new subscription

### 1.1 Specify the desired dates, product and geometry for your new subscription


In [9]:
# full API name of the desired product, in this case for Biomass Proxy v2.0
product = "BIOMASS-PROXY_V2.0_10"

# between these dates your subscription will be active. the end_date can be in the future
start_date = "2022-03-15"
end_date = "2022-09-01"

# what you would like the same of your data to be
subscription_name = "BP Test Subscription"

#path to the geometry of your field in .geojson format
field_geom_path  = open(os.path.join(sys.path[0], "Files_for_subscription_demo", "demo_field_1.geojson"))



### 1.2 Extract geometry out of geojson file
This part extracts the geometry of the geojson file provided above so that it is in the correct format for the API to handle.

In [10]:
geodf = gpd.read_file(field_geom_path)
feature = json.loads(gpd.GeoSeries(geodf.geometry).to_json())
feature_geometry = feature['features'][0]['geometry']

### 1.3 Configure the API session

In [11]:
logging.basicConfig(
        level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s"
    )
session = BaseUrlSession(base_url="https://maps.vandersat.com/api/v2/")
session.hooks["response"] = [response_hook]
session.auth = AUTH

### 1.4 Create a new subscription

Once run, you will see see from the response from the API that a subscription has been created which has a specific unique id (uuid). You should make a note of this uuid as you will need to use it to retreive the data you have requested.
#### Note if you are requesting a non-field based product (e.g. soil water content or land surface temperature, the data dictionary must contain the extra "api_requests_type" argument") - see below

In [12]:
# build a dictionary of the parameters for the subscription which can be passed to the API

data = {
        "name": subscription_name,
        "api_name": product,
        "start_date": start_date,
        "end_date": end_date,
        "geojson": feature_geometry,
        }

# IMPORTANT NOTE
# if you are creating a subscription for a non-field based product (i.e. soil moisture or LST)
# then you need to add this additional entry into the above dictionary:

      #  "arguments": {"format": "gtiff", "min_coverage": 80}


result = session.post(url="subscriptions", json=data).json()
logging.info(f"Created subscription: {result}")

# (might take a short while before the response shows below)


2023-02-08 13:09:11,893 INFO Created subscription: {'uuid': '508942b1-8d17-41a1-9d56-ea988897f933', 'created_dt': '2023-02-08T21:09:11.858312', 'modified_dt': '2023-02-08T21:09:11.777664', 'cancelled_dt': None, 'area': 140657.03, 'api_name': 'BIOMASS-PROXY_V2.0_10', 'api_request_type': 'field-based', 'name': 'BP Test Subscription', 'start_date': '2022-03-15', 'end_date': '2022-09-01', 'geojson': {'type': 'MultiPolygon', 'coordinates': [[[[-2.561268812, 56.640136001], [-2.563668605, 56.641854195], [-2.561467612, 56.644949786], [-2.555162831, 56.643018592], [-2.561268812, 56.640136001]]]]}, 'arguments': None, 'http_notify': {'url': None}}


#### Important note: only once a subscription has been made will data processing for the Biomass Proxy begin. Depending on the size of the field, subscription length, and the current processing queue, you will typically have to wait 15 mins + before the data is ready for you to download.

# Part 2 - Downloading data for an exisiting subscription

### 2.1 Configure the API session

In [13]:
logging.basicConfig(
        level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s"
    )
session = BaseUrlSession(base_url="https://maps.vandersat.com/api/v2/")
session.hooks["response"] = [response_hook]
session.auth = AUTH

### 2.2 Functions needed to download via the API
Ensure this block is ran before proceeding

In [14]:
def download_files(session: BaseUrlSession, urls: List[str], output_folder: str):
    """Save URL(s) using the Content-Disposition header's file name."""
    os.makedirs(output_folder, exist_ok=True)
    for url in urls:
        # Find existing files: assume the Content-Disposition header
        # uses the same name as the URL or at most adds a prefix, so a
        # wildcard search suffices. A real application should not rely
        # on that: use the fulfillment date or UUID to track handling.
        name = url.split("/")[-2]
        existing = glob.glob(os.path.join(output_folder, f"*{name}"))
        if existing:
            logging.info(f"Skipped existing file: name={existing[0]}; url={url}")
            continue

        r = session.get(url=url, stream=True)
        try:
            filename = stream.stream_response_to_file(r, path=output_folder)
            logging.info(f"Downloaded file: name={filename}")
        except StreamingError as e:
            logging.error(f"Failed to download file; error={str(e)}; url={url}")


def get_all_pages(
    session: BaseUrlSession, url: str, page_size: int = 50
) -> Iterator[dict]:
    """Get a generator to fetch paginated API results page by page."""
    params = {"page": 1, "limit": page_size}
    first_page = session.get(url=url, params=params).json()
    yield first_page

    page_count = math.ceil(first_page["total_items"] / page_size)
    for params["page"] in range(2, page_count + 1):
        next_page = session.get(url=url, params=params).json()
        yield next_page
        

def handle_fulfillment(
    session: BaseUrlSession, subscription_uuid: str, fulfillment: dict
) -> bool:
    """Handle a single fulfillment, like from an HTTP notification."""
    if fulfillment["status"] == "Ready":
        output_folder = os.path.join(dir_for_data_download, subscription_uuid)

        download_files(session, urls=fulfillment["files"], output_folder=output_folder)

    return fulfillment["status"] in ("Ready", "Error")


def get_subscription_fulfillments(
    session: BaseUrlSession, subscription_uuid: str
) -> bool:
    """Get fulfillments; not needed when using HTTP notifications."""
    url = f"subscriptions/{subscription_uuid}/fulfillments"
    pending = None
    for page in get_all_pages(session, url=url):
        logging.info(f"Fetched page of fulfillments: result={page}")
        for fulfillment in page["fulfillments"]:
            pending = pending or not handle_fulfillment(
                session, subscription_uuid=subscription_uuid, fulfillment=fulfillment
            )
    # True if fulfillment(s) found and all were handled, False otherwise
    return not pending


### 2.3 Define which subscription you would like to download data for
Since it takes some time for a Biomass Proxy subscription to be ready for download, here is one I made earlier for use here:

In [15]:
uuid = "401d528b-053b-45ba-b3d1-3cecc5aeb650"

### 2.4 Download the tif files of your subscription 


In [17]:
# the directory where the tiffs should be saved
dir_for_data_download = "/Users/octave.lepinard/Desktop/training_notebooks"

if not os.path.exists(dir_for_data_download):
    os.mkdir(dir_for_data_download)

while True:
    # Get the fulfillments and download the resulting files
    if get_subscription_fulfillments(session, subscription_uuid=uuid):
        break
    logging.info("Not done yet; sleeping 10 minutes")
    time.sleep(10 * 60)
    
session.close()
    

2023-02-08 13:10:59,576 INFO Fetched page of fulfillments: result={'fulfillments': [{'uuid': '06e8f9a9-8d7b-4c92-827f-e171d2620f44', 'created_dt': '2023-02-01T16:35:48.032792', 'modified_dt': '2023-02-01T16:35:48.032792', 'date': '2022-05-01', 'http_notify_count': None, 'http_notify_status': None, 'last_http_notify_dt': None, 'last_http_notify_result': None, 'last_http_notify_message': None, 'files': ['https://maps.vandersat.com/api/v2/subscriptions/401d528b-053b-45ba-b3d1-3cecc5aeb650/fulfillments/06e8f9a9-8d7b-4c92-827f-e171d2620f44/data/2022-05-01.tif/download'], 'status': 'Ready', 'status_message': None}, {'uuid': '4416d48d-19f5-470b-95d2-e4ddb12f2e58', 'created_dt': '2023-02-01T16:35:48.093085', 'modified_dt': '2023-02-01T16:35:48.093085', 'date': '2022-05-02', 'http_notify_count': None, 'http_notify_status': None, 'last_http_notify_dt': None, 'last_http_notify_result': None, 'last_http_notify_message': None, 'files': ['https://maps.vandersat.com/api/v2/subscriptions/401d528b-053b

# Part 3 - Visualize a Subscription TIF

### 3.0.1 Install required package - rasterio for plotting

In [ ]:
pip install rasterio

### 3.0.2 Import required packages

In [ ]:
import matplotlib.pyplot as plt

import rasterio

### 3.1 Specify parameters for the plot

In [ ]:
# where your tifs are saved
path_to_saved_tifs = ""

# the uuid of the subscription you would like to visualize
uuid_to_visualize = "401d528b-053b-45ba-b3d1-3cecc5aeb650"

# the product associated with this subscription that you will plot
product = "BIOMASS-PROXY_V2.0_10"

# the date you would like to visualize
date_to_visualize = "2022-05-01"

### 3.2 Visualize a Single TIF with Colorbar using Matplotlib

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.set_xlabel("Degrees Longitude")
ax.set_ylabel("Degrees Latitude")
ax.ticklabel_format(useOffset=False)

# open the tif file using Rasterio
img = rasterio.open(path_to_tif)

# plot the opened tif
plot = rasterio.plot.show(img, 
            ax=ax, 
            cmap='Greens')


retted = rasterio.plot.show(img, ax=ax, cmap='Greens')
im = retted.get_images()[0]


fig.colorbar(im, ax=ax)
